In [1]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.noise import AlphaDropout
from keras.preprocessing.text import Tokenizer

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_words = 1000
batch_size = 16
epochs = 40
plot = True


def create_network(n_dense=6,
                   dense_units=16,
                   activation='selu',
                   dropout=AlphaDropout,
                   dropout_rate=0.1,
                   kernel_initializer='lecun_normal',
                   optimizer='adam',
                   num_classes=1,
                   max_words=max_words):
    
    model = Sequential()
    model.add(Dense(dense_units, input_shape=(max_words,),
                    kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(dropout(dropout_rate))

    for i in range(n_dense - 1):
        model.add(Dense(dense_units, kernel_initializer=kernel_initializer))
        model.add(Activation(activation))
        model.add(dropout(dropout_rate))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

def create_network1(n_dense=6,
                   dense_units=16,
                   activation='selu',
                   dropout=AlphaDropout,
                   dropout_rate=0.1,
                   kernel_initializer='lecun_normal',
                   optimizer='adam',
                   num_classes=1,
                   max_words=max_words):
    
    model = Sequential()
    model.add(Dense(dense_units, input_shape=(max_words,),
                    kernel_initializer=kernel_initializer))
    model.add(Activation(activation))
    model.add(dropout(dropout_rate))

    for i in range(n_dense - 1):
        model.add(Dense(dense_units, kernel_initializer=kernel_initializer))
        model.add(Activation(activation))
        model.add(dropout(dropout_rate))

    #model.add(Dense(num_classes))
    #model.add(Activation('softmax'))
    return model

In [3]:
network2 = {
    'n_dense': 6,
    'dense_units': 16,
    'activation': 'selu',
    'dropout': AlphaDropout,
    'dropout_rate': 0.1,
    'kernel_initializer': 'lecun_normal',
    'optimizer': 'sgd'
}

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

y_train_int= y_train
y_test_int = y_test

Loading data...
8982 train sequences
2246 test sequences
46 classes


In [5]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [6]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [7]:
y_train[8]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [8]:
model1 = create_network1(num_classes=num_classes, **network2)

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from time import time
from tensorflow.python.keras import backend as K
from keras.layers import Layer
from keras.preprocessing.sequence import pad_sequences
import itertools
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Lambda,LSTM, GRU, Conv1D, Conv2D, GlobalMaxPool1D, Dense, Dropout
import random

In [10]:
class_embeddings = np.asarray([[random.random() for i in range(0,16)] for i in range(0,46)])
class_embeddings.shape

(46, 16)

In [22]:
class ManDist(Layer):
    """
    Keras Custom Layer that calculates Manhattan Distance.
    """
    
    # initialize the layer, No need to include inputs parameter!
    def __init__(self, **kwargs):
        self.result = None
        super(ManDist, self).__init__(**kwargs)

    # input_shape will automatic collect input shapes to build layer
    def build(self, input_shape):
        super(ManDist, self).build(input_shape)

    # This is where the layer's logic lives.
    def call(self, x, **kwargs):
        self.result = K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True)
        return self.result

    # return output shape
    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)

In [23]:
left_input = Input(shape=(max_words,))
right_input = Input(shape=(max_words,))

In [24]:
# Model variables
shared_model = model1

In [25]:
#TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'- embedding layer is required
#Node error -> from keras not from tf.python.keras
#Input 'b' of 'MatMul' Op has type float32 that does not match type int32 of argument 'a'. ->
malstm_distance = ManDist()([shared_model(left_input), shared_model(right_input)])
model = Model(inputs=[left_input, right_input], outputs=[malstm_distance])

In [26]:
model.compile(loss='mean_squared_error', optimizer="adam", metrics=['accuracy'])
model.summary()
shared_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 16)           17376       input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
man_dist_2 (ManDist)            (None, 1)            0           sequential_1[3][0]               
          

In [27]:
#ValueError: Error when checking target: expected man_dist_1 to have shape (1,) but got array with shape (46,)
#==> need to convert code to suit multi-class
#change target to an integer

training_start_time = time()
malstm_trained = model.fit([x_train,x_train], y_train_int, epochs=10, verbose=1)
training_end_time = time()
print("Training time finished.\n%d epochs in %12.2f" % (10,training_end_time - training_start_time))

Epoch 1/10
8982/8982 [==============================] - 2s 249us/step - loss: 127.7989 - acc: 0.0482
Epoch 2/10
8982/8982 [==============================] - 1s 138us/step - loss: 114.5921 - acc: 0.0746
Epoch 3/10
8982/8982 [==============================] - 1s 128us/step - loss: 85.0188 - acc: 0.0872
Epoch 4/10
8982/8982 [==============================] - 1s 129us/step - loss: 73.2329 - acc: 0.0983
Epoch 5/10
8982/8982 [==============================] - 1s 128us/step - loss: 66.3137 - acc: 0.1053
Epoch 6/10
8982/8982 [==============================] - 1s 130us/step - loss: 60.5356 - acc: 0.1130
Epoch 7/10
8982/8982 [==============================] - 1s 131us/step - loss: 60.0920 - acc: 0.1116
Epoch 8/10
8982/8982 [==============================] - 1s 129us/step - loss: 56.0798 - acc: 0.1096
Epoch 9/10
8982/8982 [==============================] - 1s 132us/step - loss: 53.5341 - acc: 0.1116
Epoch 10/10
8982/8982 [==============================] - 1s 128us/step - loss: 52.0598 - acc: 0.12

In [28]:
prediction = model.predict([x_test,x_test],verbose=1)
print(prediction[0:5])

2246/2246 [==============================] - 0s 149us/step
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [29]:
np.unique(prediction) #all 0s

array([0.], dtype=float32)